# Идея проекта, выборка и предобработка данных
<hr>

## Введение

```Текущая accuracy: 60-70%```

Проект представляет из себя анализатор дипфейков на основе анализа артефактов в различных зонах лица

В ходе предварительного анализа предметной области было выделено 3 наиболее перспективных для реализации метода анализа дипфейков:

- анализ движений глаз (саккад)
- анализ синзронизации речи и движений губ
- анализ артефактов

Так как анализ движений возможен только в динамике и довольно сложен в реализации, было решено реализовать метод анализа артефактов на статических изображениях (фото)

- В качестве "источника вдохновения" использовался [проект](https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb), доступный по ссылке: 

https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb

- В качестве датасета использовались фрагменты датасета [Deep Fake Detection Challenge DFDC](https://www.kaggle.com/competitions/deepfake-detection-challenge):

https://www.kaggle.com/competitions/deepfake-detection-challenge

- Про [распознанвание зон лица](https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/) можно прочесть по ссылке:

https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/

- Сжатый [датасет 300 Faces In-the-Wild Challenge (300-W)](https://ibug.doc.ic.ac.uk/resources/300-W/) для распознавания точек на лице и аннотации к нему:

https://ibug.doc.ic.ac.uk/resources/300-W/

## Пайплайн проекта

Проект работает в следующем порядке:

- datacollector.py, faceparts.py:
1. Считывание по 1 кадру из видео-датасета DFDC (5 x 10 GB, 5 x 1700 видео)
2. Распознавание 68 точек на лице с помощью OpenCV, DLib и датасета 300-W
3. Отсев фото, которые не удалось распознать (остается около 3500 фото), запись *размера* лица и бинарного значения яркости *пикселей* в 68 зонах вокруг ключевых точек в их черно-белой версии (размер зон - 8-17% от ширины лица, сейчас подбирается) в dataset (5 х 0.2-0.5 GB)
> псевдокод: ```image.to_greyscale().cut([zone_schape])```

- notebook.ipynb (до метки "Legacy"):
1. Добавление гистограмм распределения яркости к склейке датасетов
2. Добавление метрик std и noise для каждой зоны
3. Очистка от исходных массивов пикселей, запись на диск (30 MB)

- notebook2.ipynb:
1. Отсекание лишних фейков из исходного датасета (остается 1000=500*2 из 3500 фото) для избавления от перекоса (по крайней мере в процессе подбора методов, такой датасет удобнее, иначе недо/переобучение вощникает слишком часто)
2. Сбор std и avg значений для каждого из классов => определение точек, с наибольшим значением ```|avg_fake-avg_real| / (std_fake + std_real)```
> sklearn.feature_selection.SelectKBest работает плохо, т.к. не включает метрики для всего лица, которые нужны для нормализации значений по точкам

3. Выборка самых показательных features + features по всему лицу
4. Обучение моделей

> Так как 5 из 8 моделей дают сопоставимую точность, на данном этапе целесообразно работать надо подбором фичей и размерами сканируемых зон, мб фильтрацией плохих/маленьких фото, тюнинг моделей (тщательный) следует проводть позднее

## Потенциальные темы НИРов

> 1 тема != 1 НИР, темы могут быть объединены или разбиты

- процесс сборки данных: введение про проблему и методы анализа дипфейков, датасет DFDC, распознавание точек, OpenCV, DLib, датасет 300-W
- Предобработка данных: фильтрация, гистограммы и агрегационные метрики
- Подбор features, метрики метрик, метрики по всему лицу
- Выбор и тюнинг моделей, влияние размера и перекоса датасета и т.д.

На более поздних этапах (опционально):
- выстраивание вокруг модели эконсистемы: веб-приложение, комплект клиентов на React-native / чат-бот, Google OAuth, Metamask Auth
- Биллинг для приложений, ценообразование, токены (можно крипту из тестнета (Goerli, Ether testnet) ради забавы прикрутить), мониторинг (Grafana?), Docker-Kuber, балансировка и прочие умные штуки

# Обработка данных
<hr>

> 990 фото, баланс классов 1:1, размер зон - 8..17% от ширины лица (в процессе экспериментов), f1-метрика на разных моделях - 0.60..0.70

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [1]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json("../deep_fake_src/dfdc_dataframes/df_total_0_4_compact_frame_10.json")
# df = pd.read_json("../deep_fake_src/dfdc_dataframes/dataframe_compact_total_27_frame_10_lim100_offset0.json")

In [4]:
# Балансировка классов 1:1
# Инверсные метрики для полиномов

# df = df.drop(columns=['index','face_size_px'])
df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / max(1,x))

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

print(df.shape)
# df = df.reset_index()
df.head()

(990, 281)


,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
771,310,wgzrhqihqt.mp4,True,220900,17.962886,18.376313,15.712175,14.642204,14.668037,15.450066,...,17.355372,9.917355,7.024793,13.842975,0.094613,0.000005,0.019764,0.000604,1.0,1.0
3049,430,hokbnmcntx.mp4,True,198916,14.177744,7.505807,11.225003,17.594670,19.367921,26.582855,...,20.454545,17.148760,20.454545,18.181818,0.077420,0.000005,0.026754,0.000651,1.0,1.0
836,40,cgigqncgru.mp4,True,52900,18.555926,34.471873,24.656591,23.698884,20.605281,17.312571,...,35.000000,35.000000,25.000000,58.000000,0.434783,0.000019,0.017934,0.005252,1.0,1.0
562,63,fvmtnwkoab.mp4,True,94864,16.571645,26.755870,24.473817,29.936645,26.134147,26.231894,...,14.795918,17.346939,22.448980,24.489796,0.149688,0.000011,0.030394,0.001495,1.0,1.0
3204,585,osfflgpiia.mp4,True,40804,4.179282,6.545922,6.784070,12.306466,12.515155,17.040173,...,0.000000,14.000000,19.000000,10.000000,0.323498,0.000025,0.028416,0.003069,1.0,1.0


In [5]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
True,101531.523232,46.015333,531.075642,0.176697,0.290757,0.000015,0.024049,0.00319,0.996491,0.993343


In [6]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
False,92126.343434,49.182392,534.771606,0.200743,0.308146,0.000016,0.022526,0.003057,0.999456,0.998436


In [7]:
# самые ярко-различающиеся по классам features

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
pt_57_hist_noise_0.5,0.871193,0.621420,0.167340,0.249773
pt_19_hist_noise_0.5,1.641023,1.188956,0.159742,0.452067
pt_50_hist_noise_0.5,1.561743,1.141848,0.155310,0.419895
index,541.650505,397.062626,0.154028,144.587879
pt_57_hist_noise_3,16.263845,12.023012,0.149922,4.240833
pt_56_hist_noise_0.5,0.956043,0.713759,0.145097,0.242284
pt_58_hist_noise_0.5,0.914622,0.686089,0.142770,0.228534
pt_24_hist_noise_0.5,1.649462,1.266157,0.131466,0.383305
pt_20_hist_noise_0.5,1.829038,1.406564,0.130570,0.422474
pt_58_hist_noise_3,17.602637,13.561309,0.129680,4.041328


In [8]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
face_size_px_reversed,0.000016,0.000015,0.000031
overall_face_hist_std_reversed,0.003057,0.003190,0.006247
overall_face_std_reversed,0.022526,0.024049,0.046575
overall_face_hist_noise_0.5,0.200743,0.176697,0.377440
overall_face_hist_noise_3,0.308146,0.290757,0.598903
pt_7_hist_noise_0.5,0.540935,0.558245,1.099180
pt_8_hist_noise_0.5,0.507715,0.601776,1.109491
pt_9_hist_noise_0.5,0.687613,0.705521,1.393134
pt_57_hist_noise_0.5,0.871193,0.621420,1.492613
pt_10_hist_noise_0.5,0.715615,0.818659,1.534274


In [9]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
pt_57_hist_noise_0.5,0.249773,1.492613,0.167340
pt_19_hist_noise_0.5,0.452067,2.829979,0.159742
pt_50_hist_noise_0.5,0.419895,2.703591,0.155310
index,144.587879,938.713131,0.154028
pt_57_hist_noise_3,4.240833,28.286857,0.149922
pt_56_hist_noise_0.5,0.242284,1.669802,0.145097
pt_58_hist_noise_0.5,0.228534,1.600711,0.142770
pt_24_hist_noise_0.5,0.383305,2.915619,0.131466
pt_20_hist_noise_0.5,0.422474,3.235602,0.130570
pt_58_hist_noise_3,4.041328,31.163946,0.129680


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [10]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(7).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(7).index) +
    list(df_smart.filter(like='std', axis=0).head(5).index) +
    list(df_smart.filter(like='reversed', axis=0).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

print(len(features))
features

27


['overall_face_hist_noise_0.5_reversed',
 'pt_57_hist_noise_0.5',
 'pt_56_hist_noise_0.5',
 'pt_7_hist_noise_3',
 'overall_face_hist_noise_3',
 'pt_56_hist_noise_3',
 'overall_face_hist_std_reversed',
 'pt_58_hist_noise_3',
 'pt_39_hist_noise_3',
 'pt_58_hist_noise_0.5',
 'pt_20_hist_noise_0.5',
 'pt_19_hist_noise_0.5',
 'pt_39_std',
 'pt_57_std',
 'overall_face_hist_noise_3_reversed',
 'pt_57_hist_std',
 'overall_face_hist_noise_0.5',
 'pt_50_hist_noise_0.5',
 'pt_24_hist_noise_0.5',
 'pt_57_hist_noise_3',
 'pt_58_std',
 'overall_face_hist_std',
 'pt_8_hist_noise_3',
 'pt_58_hist_std',
 'overall_face_std_reversed',
 'face_size_px_reversed',
 'pt_45_hist_noise_3']

## Разбивка данных на train и test сплиты

In [11]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно?)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
771,310,True,220900,17.962886,18.376313,15.712175,14.642204,14.668037,15.450066,14.833636,...,17.355372,9.917355,7.024793,13.842975,0.094613,0.000005,0.019764,0.000604,1.0,1.0
3049,430,True,198916,14.177744,7.505807,11.225003,17.594670,19.367921,26.582855,31.754650,...,20.454545,17.148760,20.454545,18.181818,0.077420,0.000005,0.026754,0.000651,1.0,1.0
836,40,True,52900,18.555926,34.471873,24.656591,23.698884,20.605281,17.312571,17.044190,...,35.000000,35.000000,25.000000,58.000000,0.434783,0.000019,0.017934,0.005252,1.0,1.0
562,63,True,94864,16.571645,26.755870,24.473817,29.936645,26.134147,26.231894,13.584169,...,14.795918,17.346939,22.448980,24.489796,0.149688,0.000011,0.030394,0.001495,1.0,1.0
3204,585,True,40804,4.179282,6.545922,6.784070,12.306466,12.515155,17.040173,9.137855,...,0.000000,14.000000,19.000000,10.000000,0.323498,0.000025,0.028416,0.003069,1.0,1.0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [13]:
y = df_compact['fake']
X = pd.DataFrame(df_compact.drop(columns=['fake']))
# selector = SelectKBest(f_classif, k=7)
# X = selector.fit_transform(X, y)
X =  df_compact[features]

# mask = selector.get_support()
# X = X[X.columns[mask]]

# df_compact.filter(like='size', axis=0).head()

# print(df_compact[X.columns[mask]].shape)
# df_compact[X.columns[mask]].head()

pd.DataFrame(X).head()

,overall_face_hist_noise_0.5_reversed,pt_57_hist_noise_0.5,pt_56_hist_noise_0.5,pt_7_hist_noise_3,overall_face_hist_noise_3,pt_56_hist_noise_3,overall_face_hist_std_reversed,pt_58_hist_noise_3,pt_39_hist_noise_3,pt_58_hist_noise_0.5,...,pt_50_hist_noise_0.5,pt_24_hist_noise_0.5,pt_57_hist_noise_3,pt_58_std,overall_face_hist_std,pt_8_hist_noise_3,pt_58_hist_std,overall_face_std_reversed,face_size_px_reversed,pt_45_hist_noise_3
771,1.0,1.446281,2.272727,7.024793,0.094613,6.611570,0.000604,2.892562,11.157025,0.826446,...,2.272727,3.099174,4.752066,5.816738,1654.883093,5.371901,7.404321,0.019764,0.000005,15.702479
3049,1.0,1.446281,1.859504,1.239669,0.077420,10.537190,0.000651,5.371901,5.991736,1.446281,...,1.033058,2.272727,8.884298,10.913769,1536.733475,2.066116,5.699828,0.026754,0.000005,17.148760
836,1.0,0.000000,0.000000,23.000000,0.434783,17.000000,0.005252,21.000000,38.000000,0.000000,...,0.000000,0.000000,8.000000,12.502000,190.385700,26.000000,1.109155,0.017934,0.000019,34.000000
562,1.0,0.000000,0.000000,28.571429,0.149688,19.897959,0.001495,16.836735,13.775510,0.000000,...,0.000000,0.000000,20.408163,14.027739,668.804212,26.530612,2.082233,0.030394,0.000011,9.183673
3204,1.0,0.000000,0.000000,25.000000,0.323498,17.000000,0.003069,12.000000,35.000000,0.000000,...,0.000000,0.000000,9.000000,9.655729,325.874982,26.000000,1.739856,0.028416,0.000025,20.000000


In [14]:
# print(df_compact.filter(like='size', axis=1).shape)
# df_compact.filter(like='size', axis=1).head()

In [15]:
# sizes_col = X.filter(like='size', axis=1).copy()
# X = pd.concat([
#     df_compact[X.columns[mask]],
#     sizes_col
# ], axis=0)
# # X.join(df_compact.filter(like='size', axis=1), lsuffix='_caller', rsuffix='_other')
# X.head()

In [16]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=420)

In [17]:
pd.DataFrame(X_train).head()

,overall_face_hist_noise_0.5_reversed,pt_57_hist_noise_0.5,pt_56_hist_noise_0.5,pt_7_hist_noise_3,overall_face_hist_noise_3,pt_56_hist_noise_3,overall_face_hist_std_reversed,pt_58_hist_noise_3,pt_39_hist_noise_3,pt_58_hist_noise_0.5,...,pt_50_hist_noise_0.5,pt_24_hist_noise_0.5,pt_57_hist_noise_3,pt_58_std,overall_face_hist_std,pt_8_hist_noise_3,pt_58_hist_std,overall_face_std_reversed,face_size_px_reversed,pt_45_hist_noise_3
1874,1.0,1.000000,0.250000,5.250000,0.100873,3.000000,0.000908,3.750000,13.000000,1.250000,...,0.500000,2.750000,3.000000,7.071945,1101.052972,5.250000,6.285835,0.017269,0.000005,16.500000
2168,1.0,0.000000,0.000000,0.000000,0.395062,18.750000,0.003993,17.187500,23.437500,0.000000,...,0.000000,0.000000,12.500000,7.667132,250.463921,3.125000,0.918559,0.043697,0.000031,31.250000
2784,1.0,0.000000,0.000000,2.551020,0.241415,8.673469,0.002783,1.020408,27.040816,0.000000,...,0.000000,0.000000,5.102041,6.802098,359.291289,3.571429,2.630899,0.017920,0.000010,36.734694
378,1.0,0.000000,0.000000,0.206612,0.110568,1.652893,0.001289,1.446281,10.743802,0.413223,...,0.619835,4.958678,1.033058,2.039518,775.938857,5.371901,11.868903,0.021378,0.000005,9.710744
2329,1.0,2.734375,3.515625,29.296875,0.223081,12.890625,0.002955,25.000000,26.562500,5.859375,...,7.031250,1.562500,13.281250,20.187573,338.385216,4.687500,2.439903,0.014984,0.000009,12.500000


In [18]:
y_train.head()

1874    False
2168     True
2784     True
378      True
2329    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier


In [20]:
import joblib

In [21]:
def save_model(model, X_train, y_train, X_test, y_test, p=2, name='my_model.pkl'):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    joblib.dump(clf, name)

In [22]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [23]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### SGDClassifier (f1=0.60) [to be modified]

In [24]:
print_polynomial_pipeline(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="sgd.pkl"
) 

[[27 74]
 [13 84]]
               precision    recall  f1-score   support

 class "real"       0.68      0.27      0.38       101
class "fakes"       0.53      0.87      0.66        97

     accuracy                           0.56       198
    macro avg       0.60      0.57      0.52       198
 weighted avg       0.60      0.56      0.52       198



### SVC (f1=0.64) [to be modified]

In [25]:
print_polynomial_pipeline(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="svc.pkl"
) 

[[46 55]
 [23 74]]
               precision    recall  f1-score   support

 class "real"       0.67      0.46      0.54       101
class "fakes"       0.57      0.76      0.65        97

     accuracy                           0.61       198
    macro avg       0.62      0.61      0.60       198
 weighted avg       0.62      0.61      0.60       198



### LogisticRegression (f1=0.61) [to be modified]

In [26]:
print_polynomial_pipeline(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="logistic_regression.pkl"
) 

[[59 42]
 [26 71]]
               precision    recall  f1-score   support

 class "real"       0.69      0.58      0.63       101
class "fakes"       0.63      0.73      0.68        97

     accuracy                           0.66       198
    macro avg       0.66      0.66      0.66       198
 weighted avg       0.66      0.66      0.65       198



### NN - MLPClassifier (f1=0.66) [to be modified]

In [27]:
print_polynomial_pipeline(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(34,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)
save_model(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(34,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1,
    name="mlp.pkl"
) 

[[63 38]
 [41 56]]
               precision    recall  f1-score   support

 class "real"       0.61      0.62      0.61       101
class "fakes"       0.60      0.58      0.59        97

     accuracy                           0.60       198
    macro avg       0.60      0.60      0.60       198
 weighted avg       0.60      0.60      0.60       198



### RandomForestClassifier (good, f1=0.64) [to be modified] 

In [28]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="random_forest.pkl"
) 

[[56 45]
 [27 70]]
               precision    recall  f1-score   support

 class "real"       0.67      0.55      0.61       101
class "fakes"       0.61      0.72      0.66        97

     accuracy                           0.64       198
    macro avg       0.64      0.64      0.63       198
 weighted avg       0.64      0.64      0.63       198



### DecisionTreeClassifier (f1=0.59) [to be modified]

In [29]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[53 48]
 [38 59]]
               precision    recall  f1-score   support

 class "real"       0.58      0.52      0.55       101
class "fakes"       0.55      0.61      0.58        97

     accuracy                           0.57       198
    macro avg       0.57      0.57      0.57       198
 weighted avg       0.57      0.57      0.56       198



### RadiusNeighborsClassifier (f1=0.46) [уязвим к выбросам]

In [30]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=100, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[30 71]
 [25 72]]
               precision    recall  f1-score   support

 class "real"       0.55      0.30      0.38       101
class "fakes"       0.50      0.74      0.60        97

     accuracy                           0.52       198
    macro avg       0.52      0.52      0.49       198
 weighted avg       0.52      0.52      0.49       198



### KNeighborsClassifier (f1=0.69) [to be modified]

In [38]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=4, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)
save_model(
    KNeighborsClassifier(n_neighbors=4, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1,
    name="knn.pkl"
) 

[[65 36]
 [33 64]]
               precision    recall  f1-score   support

 class "real"       0.66      0.64      0.65       101
class "fakes"       0.64      0.66      0.65        97

     accuracy                           0.65       198
    macro avg       0.65      0.65      0.65       198
 weighted avg       0.65      0.65      0.65       198

